In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# 1. 각 DDoS 공격 유형별 데이터 불러오기
syn_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/SYN_Flood/syn_flood.csv')
udp_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/UDP_Flood/udp_flood.csv')
http_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/HTTP_Flood/http_flood.csv')
dns_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/DNS_Flood/dns_flood.csv')
icmp_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/ICMP_Flood/icmp_flood.csv')

# 2. 모델 학습 함수 정의
def train_model(data, features, target):
    X = data[features]
    y = data[target].astype(int)

    # SMOTE를 이용해 데이터 클래스 불균형 해결
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # 학습 데이터와 테스트 데이터로 분할
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    # RandomForest 모델 학습
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # 교차 검증 및 성능 평가
    cross_val_scores = cross_val_score(clf, X_train, y_train, cv=5)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"교차 검증 평균 정확도: {np.mean(cross_val_scores) * 100:.2f}%")
    print(f"테스트 데이터 정확도: {accuracy * 100:.2f}%")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    return clf

# 3. 각 DDoS 공격 모델 학습
print("==== SYN Flood Model Training ====")
syn_model = train_model(syn_data, ['syn_flag', 'ack_flag', 'tcp_flags_ts', 'tcp_flags_tc', 'pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'], 'ddos_detected')

print("\n==== UDP Flood Model Training ====")
udp_model = train_model(udp_data, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'], 'ddos_detected')

print("\n==== DNS Flood Model Training ====")
dns_model = train_model(dns_data, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient', 'dest_port'], 'ddos_detected')

print("\n==== HTTP Flood Model Training ====")
http_model = train_model(http_data, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'], 'ddos_detected')

print("\n==== ICMP Flood Model Training ====")
icmp_model = train_model(icmp_data, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'], 'ddos_detected')

# 4. 통합 모델 정의
class SuricataRuleRecommendationModel:
    def __init__(self, syn_model, udp_model, dns_model, http_model, icmp_model):
        self.models = {
            "SYN Flood": syn_model,
            "UDP Flood": udp_model,
            "DNS Flood": dns_model,
            "HTTP Flood": http_model,
            "ICMP Flood": icmp_model
        }
        self.rules = {
            "SYN Flood": "alert tcp any any -> any 80 (flags: S; flow: stateless; threshold: type both, track by_src, count 100, seconds 10; msg: \"Possible Syn Flood Detected\"; sid:1000001;)",
            "UDP Flood": "alert udp any any -> any any (msg:\"Possible UDP Flood Detected\"; threshold: type both, track by_src, count 1000, seconds 10; sid:1000002;)",
            "HTTP Flood": "alert http any any -> any 80 (msg:\"Possible HTTP Flood Detected\"; flow: established; threshold: type both, track by_src, count 100, seconds 10; sid:1000003;)",
            "DNS Flood": "alert udp any any -> any 53 (msg:\"Possible DNS DDoS Attack Detected\"; threshold: type both, track by_src, count 100, seconds 10; sid:1000004;)",
            "ICMP Flood": "alert icmp any any -> any any (msg:\"Possible ICMP DDoS Attack Detected\"; threshold: type both, track by_src, count 100, seconds 10; sid:1000005;)"
        }

    def recommend_rule(self, attack_type):
        """주어진 공격 유형에 대해 Suricata 규칙을 추천합니다."""
        return self.rules.get(attack_type, "No rule available for this attack type.")

    def detect_attack(self, attack_type, X):
        """해당 공격 유형에 대한 모델로 예측을 수행합니다."""
        model = self.models.get(attack_type, None)
        if model:
            return model.predict(X)
        else:
            print(f"No model found for attack type: {attack_type}")
            return None

# 5. 통합 모델 객체 생성
integrated_model = SuricataRuleRecommendationModel(syn_model, udp_model, dns_model, http_model, icmp_model)

# 6. 통합 모델 저장
with open('integrated_suricata_recommendation_model.pkl', 'wb') as f:
    pickle.dump(integrated_model, f)
print("통합된 Suricata 추천 모델이 저장되었습니다: /content/drive/MyDrive/aDDoS/Machine_Learning/integrated_suricata_recommendation_model.pkl")

# # 7. 모델 테스트 - 저장된 모델 불러오기
# model_path = '/content/drive/MyDrive/aDDoS/Machine_Learning/integrated_suricata_recommendation_model.pkl'
# with open(model_path, 'rb') as f:
#     loaded_model = pickle.load(f)

# # 예시 입력 데이터 (특징 값들)
# syn_flood_sample = np.array([[0, 1, 1, 1, 10, 20, 300, 200]])  # SYN Flood 예시
# udp_flood_sample = np.array([[50, 40, 5000, 4000]])  # UDP Flood 예시
# dns_flood_sample = np.array([[30, 20, 3000, 2000, 53]])  # DNS Flood 예시
# http_flood_sample = np.array([[100, 90, 50000, 45000]])  # HTTP Flood 예시
# icmp_flood_sample = np.array([[80, 70, 40000, 30000]])  # ICMP Flood 예시

# # 각 공격 유형에 대해 예측 수행하는 함수 수정
# def test_prediction(attack_type, sample, feature_names):
#     sample_df = pd.DataFrame(sample, columns=feature_names)
#     prediction = loaded_model.detect_attack(attack_type, sample_df)
#     print(f"Prediction for {attack_type}: {prediction}")

# # 각 공격 유형의 예시 입력 데이터와 함께 feature names 전달
# test_prediction("SYN Flood", syn_flood_sample, ['syn_flag', 'ack_flag', 'tcp_flags_ts', 'tcp_flags_tc', 'pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
# test_prediction("UDP Flood", udp_flood_sample, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
# test_prediction("DNS Flood", dns_flood_sample, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient', 'dest_port'])
# test_prediction("HTTP Flood", http_flood_sample, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
# test_prediction("ICMP Flood", icmp_flood_sample, ['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])

==== SYN Flood Model Training ====
교차 검증 평균 정확도: 97.64%
테스트 데이터 정확도: 97.75%
Confusion Matrix:
[[13598   640]
 [    5 14461]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     14238
           1       0.96      1.00      0.98     14466

    accuracy                           0.98     28704
   macro avg       0.98      0.98      0.98     28704
weighted avg       0.98      0.98      0.98     28704


==== UDP Flood Model Training ====
교차 검증 평균 정확도: 100.00%
테스트 데이터 정확도: 100.00%
Confusion Matrix:
[[39352     0]
 [    1 39695]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39352
           1       1.00      1.00      1.00     39696

    accuracy                           1.00     79048
   macro avg       1.00      1.00      1.00     79048
weighted avg       1.00      1.00      1.00     79048


==== DNS Flood Model Training ====
교차 검증 평균 정확

In [31]:
# import pandas as pd
# import numpy as np
# import pickle
# import json

# # 1. 각 DDoS 공격 유형별 데이터 불러오기
# syn_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/SYN_Flood/syn_flood.csv')
# udp_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/UDP_Flood/udp_flood.csv')
# http_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/HTTP_Flood/http_flood.csv')
# dns_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/DNS_Flood/dns_flood.csv')
# icmp_data = pd.read_csv('/content/drive/MyDrive/aDDoS/Machine_Learning/ICMP_Flood/icmp_flood.csv')

# # 2. 통합 모델 정의
# class SuricataRuleRecommendationModel:
#     def __init__(self, syn_model, udp_model, dns_model, http_model, icmp_model):
#         self.models = {
#             "SYN Flood": syn_model,
#             "UDP Flood": udp_model,
#             "DNS Flood": dns_model,
#             "HTTP Flood": http_model,
#             "ICMP Flood": icmp_model
#         }
#         self.rules = {
#             "SYN Flood": "alert tcp any any -> any 80 (flags: S; flow: stateless; threshold: type both, track by_src, count 100, seconds 10; msg: \"Possible Syn Flood Detected\"; sid:1000001;)",
#             "UDP Flood": "alert udp any any -> any any (msg:\"Possible UDP Flood Detected\"; threshold: type both, track by_src, count 1000, seconds 10; sid:1000002;)",
#             "HTTP Flood": "alert http any any -> any 80 (msg:\"Possible HTTP Flood Detected\"; flow: established; threshold: type both, track by_src, count 100, seconds 10; sid:1000003;)",
#             "DNS Flood": "alert udp any any -> any 53 (msg:\"Possible DNS DDoS Attack Detected\"; threshold: type both, track by_src, count 100, seconds 10; sid:1000004;)",
#             "ICMP Flood": "alert icmp any any -> any any (msg:\"Possible ICMP DDoS Attack Detected\"; threshold: type both, track by_src, count 100, seconds 10; sid:1000005;)"
#         }

#     def recommend_rule(self, attack_type):
#         """주어진 공격 유형에 대해 Suricata 규칙을 추천합니다."""
#         return self.rules.get(attack_type, "No rule available for this attack type.")

#     def detect_attack(self, attack_type, X, threshold=0.05):
#         """해당 공격 유형에 대한 모델로 예측 확률을 기반으로 임계값을 적용해 예측합니다."""
#         model = self.models.get(attack_type, None)
#         if model:
#             if hasattr(model, 'predict_proba'):
#                 proba = model.predict_proba(X)
#                 prediction = (proba[:, 1] >= threshold).astype(int)  # 임계값 기반 예측
#             else:
#                 prediction = model.predict(X)  # 예측 확률이 아닌 경우에는 기본 예측 사용
#             return prediction
#         else:
#             print(f"No model found for attack type: {attack_type}")
#             return None

# # 3. 탐지 데이터를 모델 입력으로 변환하는 함수
# def prepare_features_for_model(attack_type, detection_data):
#     """CSV 파일에 있는 데이터를 기반으로 모델에 적합한 형식으로 변환."""
#     if attack_type == "SYN Flood":
#         return pd.DataFrame([[int(detection_data["tcp"]["syn"]),
#                               int(detection_data["tcp"]["ack"]),
#                               0,  # tcp_flags_ts
#                               0,  # tcp_flags_tc
#                               detection_data["pkts_toserver"],
#                               detection_data["pkts_toclient"],
#                               detection_data["bytes_toserver"],
#                               detection_data["bytes_toclient"]]],
#                             columns=['syn_flag', 'ack_flag', 'tcp_flags_ts', 'tcp_flags_tc', 'pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
#     elif attack_type == "UDP Flood":
#         return pd.DataFrame([[detection_data["pkts_toserver"],
#                               detection_data["pkts_toclient"],
#                               detection_data["bytes_toserver"],
#                               detection_data["bytes_toclient"]]],
#                             columns=['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
#     elif attack_type == "HTTP Flood":
#         return pd.DataFrame([[detection_data["pkts_toserver"],
#                               detection_data["pkts_toclient"],
#                               detection_data["bytes_toserver"],
#                               detection_data["bytes_toclient"]]],
#                             columns=['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
#     elif attack_type == "DNS Flood":
#         return pd.DataFrame([[detection_data["pkts_toserver"],
#                               detection_data["pkts_toclient"],
#                               detection_data["bytes_toserver"],
#                               detection_data["bytes_toclient"],
#                               detection_data["dest_port"]]],
#                             columns=['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient', 'dest_port'])
#     elif attack_type == "ICMP Flood":
#         return pd.DataFrame([[detection_data["pkts_toserver"],
#                               detection_data["pkts_toclient"],
#                               detection_data["bytes_toserver"],
#                               detection_data["bytes_toclient"]]],
#                             columns=['pkts_toserver', 'pkts_toclient', 'bytes_toserver', 'bytes_toclient'])
#     else:
#         print(f"Unsupported attack type: {attack_type}")
#         return None

# # 4. 탐지 데이터를 받아서 공격 유형에 맞는 Suricata 룰을 추천하는 함수
# def detect_and_recommend_rule(detection_data, threshold=0.05):
#     """DDoS 탐지 후 Suricata 룰 추천"""

#     # 각 DDoS 공격 유형의 플래그를 확인
#     attack_mapping = {
#         "is_syn_flood": "SYN Flood",
#         "is_udp_flood": "UDP Flood",
#         "is_http_flood": "HTTP Flood",
#         "is_dns_flood": "DNS Flood",
#         "is_icmp_flood": "ICMP Flood"
#     }

#     for flag, attack_type in attack_mapping.items():
#         if detection_data.get(flag):
#             # 공격 유형에 맞는 모델과 데이터를 준비
#             features = prepare_features_for_model(attack_type, detection_data)

#             if features is not None:
#                 # 모델 예측 확률 출력
#                 if hasattr(loaded_model.models[attack_type], 'predict_proba'):
#                     proba = loaded_model.models[attack_type].predict_proba(features)
#                     print(f"Predicted probabilities for {attack_type}: {proba}")
#                     prediction = (proba[:, 1] >= threshold).astype(int)
#                 else:
#                     prediction = loaded_model.models[attack_type].predict(features)
#                     print(f"Predictions for {attack_type}: {prediction}")

#                 # 공격이 탐지되면 Suricata 규칙 추천
#                 if prediction[0] == 1:
#                     rule = loaded_model.recommend_rule(attack_type)
#                     print(f"Recommended Suricata rule for {attack_type}: {rule}")
#                 else:
#                     print(f"No attack detected for {attack_type}.")
#             else:
#                 print(f"Failed to prepare features for {attack_type}")

# # 5. 학습된 모델 불러오기
# model_path = '/content/drive/MyDrive/aDDoS/Machine_Learning/integrated_suricata_recommendation_model.pkl'
# with open(model_path, 'rb') as f:
#     loaded_model = pickle.load(f)

# # 6. 탐지 데이터 예측 및 Suricata 규칙 추천 테스트
# detection_data = {
#     # SYN Flood 공격에 대한 데이터 (TCP 프로토콜, SYN 플래그)
#     "src_ip": "192.168.1.1",
#     "dest_ip": "192.168.1.100",
#     "proto": "TCP",  # SYN Flood는 TCP 기반 공격
#     "tcp": {
#         "syn": True,  # SYN 플래그가 활성화됨
#         "ack": False  # ACK 플래그는 비활성화됨
#     },
#     "pkts_toserver": 100,  # SYN Flood 공격 시 패킷 수
#     "pkts_toclient": 20,  # SYN 패킷을 서버로 보내고, 응답이 적음
#     "bytes_toserver": 5000,  # 서버로 전송되는 바이트
#     "bytes_toclient": 1500,  # 클라이언트로 전송되는 바이트
#     "is_syn_flood": True,

#     # UDP Flood 공격에 대한 데이터 (UDP 프로토콜)
#     "proto": "UDP",  # UDP Flood는 UDP 기반 공격
#     "pkts_toserver": 200,  # UDP Flood 공격 시 패킷 수
#     "pkts_toclient": 50,  # UDP 응답 패킷 수
#     "bytes_toserver": 60000,  # 서버로 전송되는 바이트
#     "bytes_toclient": 10000,  # 클라이언트로 전송되는 바이트
#     "is_udp_flood": True,

#     # HTTP Flood 공격에 대한 데이터 (HTTP 프로토콜)
#     "proto": "HTTP",  # HTTP Flood는 HTTP 기반 공격
#     "pkts_toserver": 500,  # HTTP 요청의 패킷 수
#     "pkts_toclient": 300,  # HTTP 응답의 패킷 수
#     "bytes_toserver": 700000,  # 서버로 전송되는 바이트 (HTTP 요청)
#     "bytes_toclient": 500000,  # 클라이언트로 전송되는 바이트 (HTTP 응답)
#     "is_http_flood": True,

#     # DNS Flood 공격에 대한 데이터 (UDP 프로토콜, 53번 포트)
#     "proto": "UDP",  # DNS Flood는 UDP 기반 공격
#     "dest_port": 53,  # DNS 요청의 기본 포트
#     "pkts_toserver": 300,  # DNS 요청 패킷 수
#     "pkts_toclient": 20,  # DNS 응답 패킷 수
#     "bytes_toserver": 150000,  # 서버로 전송되는 바이트 (DNS 요청)
#     "bytes_toclient": 10000,  # 클라이언트로 전송되는 바이트 (DNS 응답)
#     "is_dns_flood": True,

#     # ICMP Flood 공격에 대한 데이터 (ICMP 프로토콜)
#     "proto": "ICMP",  # ICMP Flood는 ICMP 기반 공격
#     "pkts_toserver": 500,  # ICMP 요청 패킷 수
#     "pkts_toclient": 100,  # ICMP 응답 패킷 수
#     "bytes_toserver": 80000,  # 서버로 전송되는 바이트 (ICMP 요청)
#     "bytes_toclient": 20000,  # 클라이언트로 전송되는 바이트 (ICMP 응답)
#     "is_icmp_flood": True
# }

# # 탐지 데이터로 예측 수행 및 Suricata 룰 추천
# detect_and_recommend_rule(detection_data, threshold=0.05)


Predicted probabilities for SYN Flood: [[1. 0.]]
No attack detected for SYN Flood.
Predicted probabilities for UDP Flood: [[1. 0.]]
No attack detected for UDP Flood.
Predicted probabilities for HTTP Flood: [[0.97 0.03]]
No attack detected for HTTP Flood.
Predicted probabilities for DNS Flood: [[1. 0.]]
No attack detected for DNS Flood.
Predicted probabilities for ICMP Flood: [[1. 0.]]
No attack detected for ICMP Flood.
